# 15. Stacked Generalization

In [1]:
from random import seed, randrange
from math import sqrt

from functions import *

### Define functions
*Have to redo several algorithms to make compatible with learning rates, batch sizes, etc.

In [2]:
# SUBMODEL 1
# Prepare the KNN submodel
def knn_model(train):
    return train

# Calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)

# Locate neighbors for a new row
def get_neighbors(train, test_row, num_neighbors):
    distances = list()
    for train_row in train:
        dist = euclidean_distance(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors

# Make a prediction with KNN
def knn_predict(model, test_row, num_neighbors=2):
    neighbors = get_neighbors(model, test_row, num_neighbors)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
    return prediction

# SUBMODEL 2
# Make a prediction with weights
def perceptron_predict(model, row):
    activation = model[0]
    for i in range(len(row)-1):
        activation += model[i + 1] * row[i]
    return 1.0 if activation >= 0.0 else 0.0

# Estimate Perceptron weights using stochastic gradient descent
def perceptron_model(train, l_rate=0.01, n_epoch=5000):
    weights = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        for row in train:
            prediction = perceptron_predict(weights, row)
            error = row[-1] - prediction
            weights[0] = weights[0] + l_rate * error
            for i in range(len(row)-1):
                weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
    return weights

# AGGREGATOR
# Make a prediction with coefficients
def logistic_regression_predict(model, row):
    yhat = model[0]
    for i in range(len(row)-1):
        yhat += model[i + 1] * row[i]
    return 1.0 / (1.0 + exp(-yhat))

# Estimate logistic regression coefficients using stochastic gradient descent
def logistic_regression_model(train, l_rate=0.01, n_epoch=5000):
    coef = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        for row in train:
            yhat = logistic_regression_predict(coef, row)
            error = row[-1] - yhat
            coef[0] = coef[0] + l_rate * error * yhat * (1.0 - yhat)
            for i in range(len(row)-1):
                coef[i + 1] = coef[i + 1] + l_rate * error * yhat * (1.0 - yhat) * row[i]
    return coef

# Make predictions with submodels and construct a new stacked row
# Input: list of models to make predictions, list of functions to predict for each model, row of data
# Output: original input data, predictions of submodels on this data, and expected output
def to_stacked_row(models, predict_list, row):
    # Create stacked row as horizontal vector of submodel predictions and expected output
    stacked_row = list()
    # For each model, get predicted output and append to stacked row vector
    for i in range(len(models)):
        prediction = predict_list[i](models[i], row)
        stacked_row.append(prediction)
    # Append expected output to end of stacked row vector
    stacked_row.append(row[-1])
    # Append original inputs to front of stacked row vector, then return it
    return row[0:len(row)-1] + stacked_row

# Trains a list of models, constructs new stacked dataset by making predictions with each,
# trains aggregator model on the stacked set, uses submodels and aggregator to predict on test
def stacking(train, test):
    # Lists of submodels' train and predict functions
    model_list = [knn_model, perceptron_model]
    predict_list = [knn_predict, perceptron_predict]

    # Trains the submodels and adds to list
    models = list()
    for i in range(len(model_list)):
        model = model_list[i](train)
        models.append(model)

    # Iterates train data, creates stacked dataset of input + submodel prediction + expected output
    stacked_dataset = list()
    for row in train:
        stacked_row = to_stacked_row(models, predict_list, row)
        stacked_dataset.append(stacked_row)

    # Train aggregator model on (train) stacked dataset
        # Since 'improved' stack, features consist of original inputs and submodel predictions
    stacked_model = logistic_regression_model(stacked_dataset)
    # Use trained aggregator model to predict test data outputs
    predictions = list()
    for row in test:
        # Create stacked row for each test row, i.e. getting submodel predictions for each test row
        stacked_row = to_stacked_row(models, predict_list, row)
        stacked_dataset.append(stacked_row)
        prediction = logistic_regression_predict(stacked_model, stacked_row)
        # Round logit(?) output to 0 or 1
        predictions.append(round(prediction))
    return predictions

### Testing stacked generalization on Sonar dataset

In [4]:
seed(1)

# Load and prepare data
dataset = load_csv('data/sonar.all-data.csv')
# Convert string attributes to floats
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)
# Convert class output column to int
str_column_to_int(dataset, len(dataset[0])-1)

# Evaluate algorithm
# 3 folds gives ~69 observations per fold
n_folds = 3
scores = evaluate_algorithm(dataset, stacking, n_folds)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Loaded data file data/sonar.all-data.csv with 208 rows and 61 columns.
Scores: [78.26086956521739, 76.81159420289855, 69.56521739130434]
Mean Accuracy: 74.879%
